# Classificação

## SVM (Support Vector Machine)

O SVM é um classificador discriminativo formalmente definido por um hiperplano de separação. Em outras palavras, dado um conjunto de dados de treinamento classificados (aprendizado supervisionado), o algoritmo gera um hiperplano ideal que categoriza novos exemplos. No espaço bidimensional, esse hiperplano é uma linha que divide um plano em duas partes, onde cada classe fica em cada lado.

<img src="figures/svm.png" width=600 height=600 />

Para testar o classificador, utilizaremos o caso contido no arquivo ```exemplo2.csv```.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('dados/exemplo2.csv')
df.head()

,idade,conta_corrente,risco
0,21.257389,783.127911,ruim
1,21.766573,979.747521,bom
2,25.321033,1065.328054,ruim
3,23.919268,1195.758078,bom
4,21.805298,1083.764450,bom


### Utilizando o classificador SVM
O classificador SVM está presente no pacote ```sklearn``` no caminho ```sklearn.svm``` e é identificado como ```SVC```. O bloco de código abaixo, faremos o processo completo de teste do classifidar. Dessa forma, faremos a importação da função ```train_test_split``` presente no subpacote ```sklearn.model_selection```, da função ```MinMaxScaler``` do subpacote ```sklearn.preprocessing``` e da função ```accuracy_score``` presente no subpacote ```sklearn.metrics```.

In [2]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# Separação dos inputs e outputs
X = df.drop('risco', axis=1)
y = df.risco

# Normalização dos inputs
normalizador = MinMaxScaler()
X_norm = normalizador.fit_transform(X)

# Divisão dos conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

# Treinamento do classificador SVM
svc = SVC()
svc.fit(X_train, y_train)

# Cálculo da precisão
accuracy_score(y_test, svc.predict(X_test))

0.8381294964028777

Obtemos a precisão de 0.8381 (inferior ao obtido pelo KNN)
### Trabalhando com _inputs_ categóricos
Aplicaremos o classificador em um novo conjunto de dados presente no arquivo ```exemplo3.csv```

In [3]:
df2 = pd.read_csv('dados/exemplo3.csv')
df2.head()

,idade,conta_corrente,sexo,risco
0,21.257389,783.127911,masculino,ruim
1,21.766573,979.747521,feminino,bom
2,25.321033,1065.328054,feminino,bom
3,23.919268,1195.758078,feminino,bom
4,21.805298,1083.764450,feminino,bom


Nesse exemplo temos a coluna ```sexo``` identificando o sexo de cada cliente como ```masculino``` ou ```feminino```. As variáveis de _input_ de um classificador devem, necessariamente, ser do tipo numérico.

Para transformar as informações contidas na coluna ```sexo``` para o formato numérico, faremos a binarização da coluna através da classe ```OneHotEncoder```. 

Primeiro, faremos a seleção das variáveis de _input_ do tipo ```object``` utilizando o comando ```DataFrame.select_dtypes()```, passando para o parâmetro ```include``` a lista de tipos selecionados (nesse caso, o tipo ```object```).

In [5]:
X = df2.drop('risco', axis=1)
y = df2.risco

X.select_dtypes(include=['object']).head()

,sexo
0,masculino
1,feminino
2,feminino
3,feminino
4,feminino


O resultado é um ```DataFrame``` contendo somente a coluna ```sexo```.

Faremos a binarização desse ```DataFrame``` utilizando a classe ```OneHotEncoder``` presente no subpacote ```sklearn.preprocessing```. Chamaremos de ```onehot``` o objeto da classe ```OneHotEncoder```, passando os parâmetros ```sparse=False``` (para que o resultado não seja gerado no formato de uma matriz esparsa) e ```drop="first"``` (para eliminar redundância na binarização). 

Aplicaremos o operador utilizando o comando ```OneHotEncoder.fit_transform()```, atribuindo o resultado a uma variável chamada ```X_bin```.

In [82]:
from sklearn.preprocessing import OneHotEncoder

onehot = OneHotEncoder(sparse=False, drop="first")
X_bin = onehot.fit_transform(X.select_dtypes(include=['object']))
X_bin

array([[1.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

Faremos a normalização dos dados numéricos utilizando a classe ```MinMaxScaler```, criando um objeto dessa classe chamado ```mmscaler```. Esse operador recebe somente dados numéricos, fazendo-se necessária a seleção de colunas contendo somente dados desse tipo. Para tando, utilizaremos mais uma vez o comando ```DataFrame.select_dtypes()```. Mas, dessa vez, passaremos o parâmetro ```exclude=['object']```, excluindo todas as colunas do tipo ```object```. Chamaremos o ```array``` resultante de ```X_num```.

In [83]:
mmscaler = MinMaxScaler()
X_num = mmscaler.fit_transform(X.select_dtypes(exclude=['object']))
X_num

array([[0.05586473, 0.06720224],
       [0.06499436, 0.09938945],
       [0.12872564, 0.11339923],
       ...,
       [0.85424954, 0.36522222],
       [0.62401886, 0.42781034],
       [0.61473291, 0.54031447]])

Finalmente, uniremos os ```DataFrame``` ```X_bin``` e ```X_num``` em um só ```DataFrame``` chamado ```X_all```. Para tando, utilizaremos o comando ```numpy.append()``` presente no pacote ```numpy```. 

Faremos a importação do pacote dando a ele o apelido de ```np``` e aplicaremos a função ```append()```, passando como parâmetro os ```array``` ```X_num``` e ```X_bin```. O parâmetro ```axis=1``` precisa ser acrescentado à função para explicitar que a junção será feita no eixo das colunas.

In [84]:
import numpy as np

X_all = np.append(X_num, X_bin, axis=1)
X_all

array([[0.05586473, 0.06720224, 1.        ],
       [0.06499436, 0.09938945, 0.        ],
       [0.12872564, 0.11339923, 0.        ],
       ...,
       [0.85424954, 0.36522222, 1.        ],
       [0.62401886, 0.42781034, 1.        ],
       [0.61473291, 0.54031447, 1.        ]])

Finalmente, podemos testar o classificador com o processo de validação abaixo.  

In [86]:
# Divisão dos conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_all, y, test_size=1/3, random_state=42)

# Treinamento do classificador SVM
svc2 = SVC()
svc2.fit(X_train, y_train)

# Cálculo da precisão
accuracy_score(y_test, svc2.predict(X_test))

0.7146282973621103

Para utilizar o classificar para prever a classe de um novo conjunto de dados, criaremos um ```DataFrame``` contendo cinco novos clientes com as seguintes informações:

|idade|conta_corrente|sexo|
|-|-|-|
|20|800|masculino|
|25|4000|feminino|
|50|2200|masculino|
|35|3200|feminino|
|75|1000|masculino|

Para isso, utilizaremos o comando ```pandas.DataFrame()```, passando como parâmetro o dicionário contendo como chave, os nomes das colunas e como valores, a lista de valores. Chamaremos o novo ```DataFrame```de ```df_new```.

In [94]:
df_new = pd.DataFrame({'idade': [20, 25, 50, 35, 75], 
                      'conta_corrente': [800, 4000, 2200, 3200, 1000], 
                      'sexo': ['masculino', 'feminino', 'masculino', 'feminino', 'feminino']})
df_new

,idade,conta_corrente,sexo
0,20,800,masculino
1,25,4000,feminino
2,50,2200,masculino
3,35,3200,feminino
4,75,1000,feminino


Para fazer a previsão do risco de cada um dos novos clientes, precisamos transformar o ```DataFrame``` para o mesmo formato utilizado no treinamento do classificador.

In [95]:
X_new_bin = onehot.transform(df_new.select_dtypes(include=['object']))
X_new_num = mmscaler.transform(df_new.select_dtypes(exclude=['object']))
X_new = np.append(X_new_num, X_new_bin, axis=1)
svc2.predict(X_new)

array(['ruim', 'bom', 'ruim', 'bom', 'bom'], dtype=object)

Para mostrar o resultado no formato de um ```DataFrame```, basta criarmos uma nova coluna contendo os resultados da previsão.

In [97]:
df_previsao = df_new.copy()
df_previsao['previsao'] = svc2.predict(X_new)
df_previsao

,idade,conta_corrente,sexo,previsao
0,20,800,masculino,ruim
1,25,4000,feminino,bom
2,50,2200,masculino,ruim
3,35,3200,feminino,bom
4,75,1000,feminino,bom
